In [17]:
import pandas as pd 
df=pd.read_csv("filtered_dataset2.csv")

In [18]:
import re
def clean_description(text):
    if pd.isna(text):
        return ""
    return re.sub(r'[\n\r\t\/]+', ' ', text).strip()
df['description'] = df['description'].apply(clean_description)
df = df[df['description'].str.strip() != '']

In [19]:
def split_entity_value(value):
    match = re.match(r'(\d+\.?\d*)\s*(\w+)', value)  # Matches number followed by a unit
    if match:
        return match.group(1), match.group(2)  # Return number and unit as a tuple
    return None, None  # Return None if no match is found

# Apply the function to the entity_value column
df['value'], df['unit'] = zip(*df['entity_value'].apply(split_entity_value))
df.dropna(inplace=True)

In [20]:
df.dropna(inplace=True)
df['description'] = df['description'].fillna('')

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, f1_score
import scipy.sparse as sp
import xgboost as xgb
# Feature Engineering
tfidf = TfidfVectorizer()
X_description = tfidf.fit_transform(df['description'])
label_encoder = LabelEncoder()
X_entity_name = label_encoder.fit_transform(df['entity_name']).reshape(-1, 1)

# Combine features
X_entity_name_sparse = sp.csr_matrix(X_entity_name)
X= sp.hstack((X_description, X_entity_name_sparse))

# Prepare target variables
y_value = df['value'].astype(float)
y_unit = df['unit']

# Train-test split
X_train, X_test, y_value_train, y_value_test, y_unit_train, y_unit_test = train_test_split(
    X, y_value, y_unit, test_size=0.2, random_state=42)

# Model for value prediction
model_value = RandomForestRegressor()
model_value.fit(X_train, y_value_train)
y_value_pred = model_value.predict(X_test)

y_encoded = label_encoder.fit_transform(y_unit_train)
# Model for unit prediction
model_unit =xgb.XGBClassifier(
    objective='multi:softmax',  # Multi-class classification
    num_class=len(label_encoder.classes_),  # Number of unique classes
    use_label_encoder=False,
    eval_metric='mlogloss'
)

model_unit.fit(X_train, y_encoded)
y_unit_pred = model_unit.predict(X_test)
y_unit_pred_original = label_encoder.inverse_transform(y_unit_pred)
f1 = f1_score(y_unit_test, y_unit_pred_original, average='weighted')
print(f'F1 Score: {f1}')


/home/mars/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:37:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1 Score: 0.7431095862827195
